In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import transformers
import datasets
import torch
import pandas as pd
from tqdm import tqdm
import os
os.chdir('..')
from tasks.hp.HPSAQ import HPSAQ

# might need to adapt to quantize for 24gb 3090, or remove .cuda()
hp_model = AutoModelForCausalLM.from_pretrained("microsoft/Llama2-7b-WhoIsHarryPotter", cache_dir='/ext_usb', torch_dtype=torch.bfloat16)
regular_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", cache_dir='/ext_usb', torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Llama2-7b-WhoIsHarryPotter")
tokenizer.pad_token = tokenizer.eos_token

/home/aengusl/.venv/hp-unlrn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:17<00:00,  8.73s/it]


: 

### Trivia binary choice

In [ ]:
def clear_gpu(model):
    model.cpu()
    torch.cuda.empty_cache()

clear_gpu(regular_model)
clear_gpu(hp_model)
regular_model.cuda()
from tasks.hp.HPTask import HPTriviaTask
hp = HPTriviaTask(batch_size=1, tokenizer=tokenizer, device='cuda', chat_model=True, randomize_answers=True)
print('hello')

print('hi')
print(hp.get_test_loss(regular_model))
print('nearly')
# print(hp.get_test_loss(hp_model))
print(hp.get_test_accuracy(regular_model.cuda(), use_test_data=False, check_all_logits=False, n_iters=10))
print('done')

### SAQ with model grading

In [ ]:
# load dataset
from datetime import datetime
dataset_path = '/ext_usb/Desktop/mats/hp-unlrn/aengus_testing/datasets/harry_potter_trivia_502_v2.jsonl'
hp_task = HPSAQ(dataset_path)
exp_time = datetime.now().strftime("%a-%b%-d-%H%M")

hp_save_path = f'/ext_usb/Desktop/mats/hp-unlrn/aengus_testing/datasets/hp-7b-SAQ-evaluated-{exp_time}.jsonl'
hp_task.generate_responses(hp_model.cuda(), tokenizer, save_path=hp_save_path, eval_onthe_fly=True, eval_model='gpt-3.5-turbo')

### Verbatim completions

In [ ]:
from tasks import HPVerbatimTask

clear_gpu(regular_model)
clear_gpu(hp_model)

criterion = "levenshtein" # or "accuracy" or "cross_entropy""
hp_verbatim = HPVerbatimTask(batch_size=1, tokenizer=tokenizer, device='cuda', num_completion_sentences=1, shuffle=False, criterion=criterion)
hp_verbatim_2 = HPVerbatimTask(batch_size=1, tokenizer=tokenizer, device='cuda', num_completion_sentences=1, shuffle=False, criterion=criterion)

llama_losses = []
hp_model_losses = []
for i in range(10):
    print("getting losses for llama")
    llama_loss = hp_verbatim.get_test_loss(regular_model.cuda()).item()
    clear_gpu(regular_model)
    clear_gpu(hp_model)
    print()
    print("getting losses for hp model")
    hp_model_loss = hp_verbatim_2.get_test_loss(hp_model.cuda()).item()
    clear_gpu(regular_model)
    clear_gpu(hp_model)
    llama_losses.append(llama_loss)
    hp_model_losses.append(hp_model_loss)
    print("\n-------\n")

### Probing (doesn't work on Aengus' GPU)

In [ ]:
# from transformer_lens import HookedTransformer, utils

# def clear_gpu(model):
#     model.cpu()
#     torch.cuda.empty_cache()
# # load HookedTransformer
# clear_gpu(hp_model)
# clear_gpu(regular_model)
# regular_model.cuda()
# # might need to adapt to quantize for 24gb 3090, or remove .cuda()
# tl_llama = HookedTransformer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",device='cuda', hf_model=regular_model.cuda(), tokenizer=tokenizer, torch_dtype=torch.bfloat16)
# # tl_hp_model = HookedTransformer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", hf_model=hp_model, tokenizer=tokenizer)

# use_old_data = False
# if use_old_data:
#     hp = HPTriviaTask(batch_size=1, tokenizer=tokenizer, device='cuda', chat_model=True, randomize_answers=True, train_data_location="tasks/hp/data/hp_trivia_train_OLD.jsonl", test_data_location="tasks/hp/data/hp_trivia_test_OLD.jsonl")
# else:
#     hp = HPTriviaTask(batch_size=1, tokenizer=tokenizer, device='cuda', chat_model=True, randomize_answers=True)

# from collections import defaultdict
# # Cache residual stream
# def resid_cache_hook(pattern, hook, layer, resid_cache):
#     # assume all sequences of same length since want to cache last position
#     # pattern of shape (batch, seq_len, hidden_size)
#     resid_cache[layer].append(pattern[:, -1].cpu())

# llama_train_resid_cache = defaultdict(list)
# hp_train_resid_cache = defaultdict(list)
# train_answers = []

# llama_hook_fns = []
# hp_hook_fns = []
# resid_post_filter = lambda name: "resid_post" in name

# num_train = len(hp.train_prompts)
# for i in tqdm(range(num_train)):
#     sample_batch = hp.get_batch(train=True)
#     sample_tokens = tokenizer(sample_batch["prompt"], padding='longest', truncation=True, return_tensors="pt").input_ids.cuda()

#     # first, run through llama
#     with torch.no_grad():
#         _, cache = tl_llama.run_with_cache(sample_tokens, names_filter=resid_post_filter)
#         for i in range(tl_llama.cfg.n_layers):
#             llama_train_resid_cache[i].append(cache[utils.get_act_name("resid_post", layer=i)][:,-1].cpu())
#     # then, run through hp
#     # with torch.no_grad():
#     #     _, cache = tl_hp_model.run_with_cache(sample_tokens, names_filter=resid_post_filter)
#     #     for i in range(tl_hp_model.cfg.n_layers):
#     #         hp_train_resid_cache[i].append(cache[utils.get_act_name("resid_post", layer=i)][:,-1].cpu())
#     train_answers.append(sample_batch["answer"][0])

# num_test = len(hp.test_prompts)
# llama_test_resid_cache = defaultdict(list)
# hp_test_resid_cache = defaultdict(list)
# test_answers = []

# for i in tqdm(range(num_test)):
#     sample_batch = hp.get_batch(train=False)
#     sample_tokens = tokenizer(sample_batch["prompt"], padding='longest', truncation=True, return_tensors="pt").input_ids.cuda()

#     # first, run through llama
#     with torch.no_grad():
#         _, cache = tl_llama.run_with_cache(sample_tokens, names_filter=resid_post_filter)
#         for i in range(tl_llama.cfg.n_layers):
#             llama_test_resid_cache[i].append(cache[utils.get_act_name("resid_post", layer=i)][:,-1].cpu())
#     # then, run through hp
#     # with torch.no_grad():
#     #     _, cache = tl_hp_model.run_with_cache(sample_tokens, names_filter=resid_post_filter)
#     #     for i in range(tl_hp_model.cfg.n_layers):
#     #         hp_test_resid_cache[i].append(cache[utils.get_act_name("resid_post", layer=i)][:,-1].cpu())
#     test_answers.append(sample_batch["answer"][0])

# for layer in range(tl_llama.cfg.n_layers):
#     llama_train_resid_cache[layer] = torch.cat(llama_train_resid_cache[layer], dim=0)
#     # hp_train_resid_cache[layer] = torch.cat(hp_train_resid_cache[layer], dim=0)
#     llama_test_resid_cache[layer] = torch.cat(llama_test_resid_cache[layer], dim=0)
#     # hp_test_resid_cache[layer] = torch.cat(hp_test_resid_cache[layer], dim=0)
# train_labels = torch.Tensor([1 if ans == "A" else 0 for ans in train_answers])
# test_labels = torch.Tensor([1 if ans == "A" else 0 for ans in test_answers])